# Challenge 1

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
def is_running_on_kaggle():
    return "KAGGLE_KERNEL_RUN_TYPE" in os.environ and os.environ["KAGGLE_KERNEL_RUN_TYPE"] == "Interactive"
data_path = '/kaggle/input/' if is_running_on_kaggle() else 'data/'

#for dirname, _, filenames in os.walk(data_path):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))
#
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ImportError: Unable to import required dependencies:
pytz: No module named 'pytz'

In [6]:
import torch
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor

# Imports
import warnings
import matplotlib.pyplot as plt
from torchvision.io import read_image

class DatasetLoader(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [13]:
annotations_file = data_path + 'train.csv'
img_dir = data_path + 'train/train/'
training_data = DatasetLoader(annotations_file, img_dir)

print(training_data[1][0].shape)
print(training_data[1])

torch.Size([3, 32, 32])
(tensor([[[118, 108, 104,  ..., 158, 138, 123],
         [133, 123, 117,  ..., 135, 119, 116],
         [150, 125, 107,  ...,  99, 103, 125],
         ...,
         [205, 212, 225,  ..., 185, 208, 245],
         [111, 120, 129,  ..., 113, 142, 185],
         [ 93,  67,  60,  ..., 104,  97, 126]],

        [[109,  99,  95,  ..., 132, 112,  97],
         [124, 114, 108,  ..., 109,  93,  90],
         [141, 116,  98,  ...,  73,  77,  99],
         ...,
         [190, 197, 213,  ..., 173, 195, 232],
         [ 96, 105, 117,  ..., 100, 126, 169],
         [ 78,  52,  48,  ...,  91,  81, 110]],

        [[114, 104, 100,  ..., 143, 123, 108],
         [129, 119, 113,  ..., 120, 104, 101],
         [144, 119, 103,  ...,  86,  90, 112],
         ...,
         [195, 202, 217,  ..., 183, 205, 242],
         [101, 110, 121,  ..., 110, 137, 180],
         [ 83,  57,  52,  ..., 101,  92, 121]]], dtype=torch.uint8), 1)


In [1]:
class TinyModel(torch.nn.Module):

    def __init__(self):
        super(TinyModel, self).__init__()

        self.linear1 = torch.nn.Linear(100, 200)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(200, 10)
        self.softmax = torch.nn.Softmax()

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x

tinymodel = TinyModel()

print('The model:')
print(tinymodel)

NameError: name 'torch' is not defined